In [1]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/캡스톤/data/food_3.zip"

In [5]:
with open("./food_3/train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open("./food_3/valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open("./food_3/test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [6]:
with open("./food_3/train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open("./food_3/valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open("./food_3/test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [7]:
len(train_target)+len(valid_target)+len(test_target)

130524

## Model

In [8]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
#current_path ='data/'
#local_path = 'data/'

In [9]:
# set parameters:
# V = 142381
# V = 226928
V = 74258  #33531 # len(tokenizer.word_index) #249274
embedding_dim = 50
max_len =  1   # 1945  # maxlen
filters_num = 64
kernel_size = 3

In [10]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))

input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu',
                padding='same'
                )
maxpool1d = MaxPool1D(max_len)
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(conv1d)(input_vector)
validation_conv = conv1d(validation_vector)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [11]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 3, 1, 50) (None, 1, 50) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [12]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [13]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [14]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mae', metrics=['mse', 'mae', 'mape'])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1, 50)    3712900     ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        3712900     ['input_2[0][0]']            

In [16]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 16s 11ms/step - loss: 1.2943 - mse: 4.1530 - mae: 1.2943 - mape: 56.8129 - val_loss: 0.7893 - val_mse: 2.1762 - val_mae: 0.7893 - val_mape: 50.0624
Epoch 2/30
256/256 [==============================] - 2s 10ms/step - loss: 0.7719 - mse: 1.9458 - mae: 0.7719 - mape: 47.7747 - val_loss: 0.8071 - val_mse: 1.9984 - val_mae: 0.8071 - val_mape: 47.5974
Epoch 3/30
256/256 [==============================] - 2s 10ms/step - loss: 0.7150 - mse: 1.7425 - mae: 0.7150 - mape: 43.2260 - val_loss: 0.8290 - val_mse: 1.9675 - val_mae: 0.8290 - val_mape: 46.7607
Epoch 4/30
256/256 [==============================] - 2s 10ms/step - loss: 0.6921 - mse: 1.6799 - mae: 0.6921 - mape: 41.7689 - val_loss: 0.8367 - val_mse: 1.9936 - val_mae: 0.8367 - val_mape: 46.4817
Epoch 5/30
256/256 [==============================] - 2s 9ms/step - loss: 0.6800 - mse: 1.6431 - mae: 0.6800 - mape: 40.8977 - val_loss: 0.8439 - val_mse: 2.0019 - val_mae: 0.8439 - val_mape: 46.

In [17]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 7s 3ms/step - loss: 0.9176 - mse: 2.0562 - mae: 0.9176 - mape: 47.1971
[0.9175692796707153, 2.056164264678955, 0.9175692796707153, 47.19711685180664]


In [18]:
#model.save(path+"IMDB_ADLFM_baseline")